In [ ]:
%matplotlib inline

import scipy.io
import matplotlib.pyplot as plt
import scipy
import numpy as np
import os
import itertools

In [ ]:
def flatMap(f, items):
        return list(itertools.chain.from_iterable(itertools.imap(f, items)))

In [ ]:
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len/2-1):-(window_len/2)]


In [ ]:
def get_cell_type(cell):
    num = cell['CellNum'][0][0][0][0]
    if num in [19,20,21,22,23,24,36,37,38]:
        return 'SpVi'
    else:
        return 'PrV'

In [ ]:
NOPRINT_LEVEL = 0
NORMAL_PRINT_LEVEL = 1
DEBUG_LEVEL = 2
PRINT_LEVEL = NORMAL_PRINT_LEVEL

In [ ]:
for filename in [fn for fn in os.walk('data').next()[2] if fn.endswith('.mat')]:
    filepath = os.path.join('data', filename)
    cellNumber = int(filename[5:7])
    
    if(PRINT_LEVEL == NORMAL_PRINT_LEVEL): print 'Outputting raw graphs for cell #%d in file: %s' % (cellNumber, filename)
    
    mat = scipy.io.loadmat(filepath)
    cell = mat['Cell']
    vMat = cell['vMat'][0][0].transpose()
    stimMat = cell['stimMat'][0][0].transpose()
    xValuesVec = [x[0] for x in cell['xValue'][0][0].tolist()]
    
    for i in range(len(vMat)):
        xValue = xValuesVec[i]
        
        
        outdir = r".\data\graphs\all trials\raw\%s\Cell0%d" % (get_cell_type(cell), cellNumber)
        stim_outpath = os.path.join(outdir, (r"Cell0%d_stim_%d_%d.pdf" % (cellNumber, i, (xValue+1))))
        volt_outpath = os.path.join(outdir, (r"Cell0%d_volt_%d_%d.pdf" % (cellNumber, i, (xValue+1))))
        
        try:
            os.makedirs(outdir)
        except:
            pass
        
        if(PRINT_LEVEL == DEBUG_LEVEL): print '\tOutputting trial %d with xValue %d to %s and %s' % (i, xValue, stim_outpath, volt_outpath)
            
        plt.close(1)
        plt.rc('xtick', labelsize= 2.65)
        plt.plot([x * 0.25 for x in range(0, len(stimMat[i]))], stimMat[i])
        plt.xticks(np.arange(0, (len(stimMat[i])+1)*0.25, 50.0))
        plt.savefig(stim_outpath)
        plt.close(1)
    
        plt.close(1)
        plt.rc('xtick', labelsize=2.65)
        plt.plot([x * 0.05 for x in range(0, len(vMat[i]))], vMat[i])
        plt.xticks(np.arange(0, (len(vMat[i])+1)*0.05, 50.0))        
        plt.savefig(volt_outpath)
        plt.close(1)
        
print "Done outputting raw graphs"

In [ ]:
for filename in [fn for fn in os.walk('data').next()[2] if fn.endswith('.mat')]:
    filepath = os.path.join('data', filename)
    cellNumber = int(filename[5:7])
    
    if(PRINT_LEVEL == NORMAL_PRINT_LEVEL): print 'Outputting smooth graphs for cell #%d in file: %s' % (cellNumber, filename)
    
    mat = scipy.io.loadmat(filepath)
    cell = mat['Cell']
    vMat = cell['vMat'][0][0].transpose()
    stimMat = cell['stimMat'][0][0].transpose()
    xValuesVec = [x[0] for x in cell['xValue'][0][0].tolist()]
    
    for i in range(len(vMat)):
        xValue = xValuesVec[i]
        
        outdir = r".\data\graphs\all trials\smooth\%s\Cell0%d" % (get_cell_type(cell), cellNumber)
        stim_outpath = os.path.join(outdir, (r"Cell0%d_stim_%d_%d.pdf" % (cellNumber, i, (xValue+1))))
        volt_outpath = os.path.join(outdir, (r"Cell0%d_volt_%d_%d.pdf" % (cellNumber, i, (xValue+1))))
        
        smooth_stim_vec = smooth(stimMat[i], window_len = 100,window='hanning')
        smooth_volt_vec = smooth(vMat[i], window_len = 100,window='hanning')
        
        try:
            os.makedirs(outdir)
        except:
            pass
            
        if(PRINT_LEVEL == DEBUG_LEVEL): print '\tOutputting trial %d with xValue %d to %s and %s' % (i, xValue, stim_outpath, volt_outpath)            
        
        plt.close(1)
        plt.rc('xtick', labelsize= 2.65)
        
        plt.plot([x * 0.25 for x in range(0, len(smooth_stim_vec))], smooth_stim_vec)
        plt.xticks(np.arange(0, (len(smooth_stim_vec)+1)*0.25, 50.0))
        plt.savefig(stim_outpath)
        plt.close(1)
    
        plt.close(1)
        plt.rc('xtick', labelsize=2.65)
        plt.plot([x * 0.05 for x in range(0, len(smooth_volt_vec))], smooth_volt_vec)
        plt.xticks(np.arange(0, (len(smooth_volt_vec)+1)*0.05, 50.0))        
        plt.savefig(volt_outpath)
        plt.close(1)
        
print "Done outputting smooth graphs"